In [1]:
import os  
import glob
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix


import PIL 
import random
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torchinfo import summary 

import torch.optim as optim
from IPython.display import Image
from torch.utils.data import DataLoader, Dataset

from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.nn.functional as F

import pydicom
from PIL import Image

import cv2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda')

Set Random Seed for reproducability

In [3]:
random_seed = 124
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

In [4]:
train_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/train_df.csv')
test_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/test_df.csv')
valid_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/valid_df.csv')

In [5]:
train_df = train_df[['Image Index', 'Finding Labels', 'Path Image']]
test_df = test_df[['Image Index', 'Finding Labels', 'Path Image']]
valid_df = valid_df[['Image Index', 'Finding Labels', 'Path Image']]

In [6]:
labels = ['No Finding', 'Infiltration', 'Atelectasis', 'Effusion', 'Nodule', 'Pneumothorax', 'Mass']
labels_dict = {'No Finding': 0, 'Infiltration': 1, 'Atelectasis': 2, 'Effusion': 3, 'Nodule': 4, 'Pneumothorax': 5, 'Mass': 6}

In [7]:
train_path = list(train_df['Path Image'])
test_path = list(test_df['Path Image'])
valid_path = list(valid_df['Path Image'])

In [8]:
train_label = [labels_dict[item] if item in labels_dict else item for item in list(train_df['Finding Labels'])]
test_label = [labels_dict[item] if item in labels_dict else item for item in list(test_df['Finding Labels'])]
valid_label = [labels_dict[item] if item in labels_dict else item for item in list(valid_df['Finding Labels'])]

In [9]:
sum(valid_df['Finding Labels'] == labels[0])

1500

In [10]:
df = pd.DataFrame(train_label)

for i in range(0,len(labels),1):
    print(sum(train_df['Finding Labels'] == labels[i]) ==  sum(df[0] == i))

True
True
True
True
True
True
True


In [11]:
len(labels)

7

In [12]:
type(train_label), type(test_label), type(valid_label)


(list, list, list)

In [13]:
train_df.head(1)

,Image Index,Finding Labels,Path Image
0,00000005_006.png,Infiltration,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...


In [14]:
train_label[:5]

[1, 1, 1, 1, 1]

### Dataset Class



In [15]:
class CT_Dataset(Dataset):
    def __init__(self, img_path, img_labels, img_transforms=None, grayscale=False):
        self.img_path = img_path
        self.img_labels = torch.Tensor(img_labels)
        if (img_transforms is None) & (grayscale == True):
            self.transforms = transforms.Compose([transforms.Grayscale(),
                                                  transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        elif grayscale == False:
            self.transforms = transforms.Compose([transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        else:
            self.transforms = img_transforms
    
    def __getitem__(self, index):
        # load image
        cur_path = self.img_path[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transforms(cur_img)

        return cur_img, self.img_labels[index]
    
    def __len__(self):
        return len(self.img_path)

## 2. Model Development

### ResNet50

In [16]:

class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        in_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet50(x)
    
    def get_name(self):
        return 'ResNet50'


### ResNet101

In [17]:
class ResNet101(nn.Module):
    def __init__(self, num_classes):
        super(ResNet101, self).__init__()
        self.resnet101 = models.resnet101(pretrained=True)
        in_features = self.resnet101.fc.in_features
        self.resnet101.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet101(x)
    
    def get_name(self):
        return 'ResNet101'


### EfficientNetB0

In [18]:
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB0, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b0')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)
    
    def get_name(self):
        return 'EfficientNetB0'


### EfficientNetB4

In [19]:
class EfficientNetB4(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB4, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b4')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)
    
    def get_name(self):
        return 'EfficientNetB4'

### EfficientNetB7

In [20]:
class EfficientNetB7(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB7, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b7')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)
    
    def get_name(self):
        return 'EfficientNetB7'

### VGG16

In [21]:
class VGG16(nn.Module):
    def __init__(self, num_classes):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        in_features = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.vgg16(x)
    
    def get_name(self):
        return 'VGG16'

### Function train_model

In [22]:
result_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/result_df.csv')
qnt_labels = result_df['Finding Labels'].value_counts()
qnt_images_each_label = qnt_labels.to_dict()
qnt_images_each_label

{'No Finding': 15000,
 'Infiltration': 9547,
 'Atelectasis': 4215,
 'Effusion': 3955,
 'Nodule': 2705,
 'Pneumothorax': 2194,
 'Mass': 2139}

In [23]:
total_samples = 0
for value in qnt_images_each_label.values():
    total_samples += value

In [24]:
qnt_images_each_label['Infiltration']

9547

In [25]:
labels

['No Finding',
 'Infiltration',
 'Atelectasis',
 'Effusion',
 'Nodule',
 'Pneumothorax',
 'Mass']

In [27]:
num_classes = len(train_df['Finding Labels'].unique())

weight_No_Finding = total_samples/(qnt_images_each_label['No Finding'] * num_classes)
weight_Infiltration = total_samples/( qnt_images_each_label['Infiltration'] * num_classes)
weight_Atelectasis = total_samples/( qnt_images_each_label['Atelectasis'] * num_classes)
weight_Effusion = total_samples/( qnt_images_each_label['Effusion'] * num_classes)
weight_Nodule = total_samples/( qnt_images_each_label['Nodule'] * num_classes)
weight_Pneumothorax = total_samples/( qnt_images_each_label['Pneumothorax'] * num_classes)
weight_Mass = total_samples/( qnt_images_each_label['Mass'] * num_classes)

weights = [weight_No_Finding, weight_Infiltration, weight_Atelectasis, weight_Effusion, weight_Nodule, weight_Pneumothorax, weight_Mass]
weights

[0.37861904761904763,
 0.5948764757814721,
 1.3473987459752583,
 1.4359761603756547,
 2.0995510958542383,
 2.5885531970308633,
 2.6551125358979495]

In [28]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

# Define the training function
def train_model(model, train_dataset, val_dataset, test_dataset, device, path_save_model, 
                lr=0.0001, epochs=30, batch_size=32, l2=0.00001, gamma=0.5,
                patience=7):
    model = model.to(device)

    # Construct dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # History
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    weight = torch.tensor(weights).to(device)
    
    # Set up loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight=weight)  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=patience, gamma=gamma)

    # Initialize variables to track the best validation loss and corresponding model state
    best_val_loss = float('inf')
    best_model_state = None

    # Training Loop
    print("Training Start:")
    for epoch in range(epochs):
        model.train()

        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device).long()   # Convert labels to Long data type

            outputs = model(images).float()  # Make sure the output is of type float
            pred = outputs.argmax(dim=1)  ####################################################
            # print(f'Pred\n{pred}')
            # print(f'labels\n{labels}')
            # print(f'outputs\n{outputs}')
            cur_train_loss = criterion(outputs, labels)
            # pred = torch.nn.functional.one_hot(pred, num_classes=num_classes).cpu().numpy()

            cur_train_acc = (pred == labels).sum().item() / batch_size

            cur_train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss += cur_train_loss 
            train_acc += cur_train_acc
        
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device).long()   # Convert labels to Long data type

                outputs = model(images).float()  # Make sure the output is of type float

                cur_valid_loss = criterion(outputs, labels)
                val_loss += cur_valid_loss

                pred = outputs.argmax(dim=1)  ####################################################
                # pred = torch.nn.functional.one_hot(pred, num_classes=num_classes).cpu().numpy()
                val_acc += (pred == labels).sum().item() / batch_size

        scheduler.step()

        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        val_loss = val_loss / len(val_loader)
        val_acc = val_acc / len(val_loader)

        print(f"Epoch:{epoch + 1} / {epochs}, lr: {optimizer.param_groups[0]['lr']:.5f} train loss:{train_loss:.5f}, train acc: {train_acc:.5f}, valid loss:{val_loss:.5f}, valid acc:{val_acc:.5f}")
    
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
    
        # Update the best model if validation loss is the lowest so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()

    # Load the best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    test_acc = 0
    print(f'The best val loss is {best_val_loss}.\n\n')
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).long()   # Convert labels to Long data type

            outputs = model(images).float()  # Make sure the output is of type float
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            pred = outputs.argmax(dim=1)  ####################################################
            # pred = torch.nn.functional.one_hot(pred, num_classes=num_classes).cpu().numpy()
            test_acc += (pred == labels).sum().item()

    y_true = np.array(y_true)
    y_pred = np.array(pred)
    
        # Calculate evaluation metrics
    accuracy = (test_acc / len(test_loader))
    tn, fp, fn, tp = multilabel_confusion_matrix(y_true, y_pred).ravel()
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    specificity = tn / (tn + fp)
    sensitivity = tp / (fn + tp)

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'specificity': specificity,
        'sensitivity': sensitivity,
        'Best Value Loss': best_val_loss,
        'false negativo': fn
    }
    
    dataset = pd.DataFrame(metrics.items(), columns=['Metric', 'Value']).to_csv(f'{path_save_model}.csv')
    print(f'Test Accuracy:  {accuracy}\n')
    print(f'Metrics:  {metrics}\n')

    return history, model


Process the datasets and train the model

In [29]:
print("Current GPU memory usage:", torch.cuda.memory_allocated() / (1024 ** 2), "MB")
print("Max GPU memory usage:", torch.cuda.max_memory_allocated() / (1024 ** 2), "MB")

torch.cuda.empty_cache()

Current GPU memory usage: 0.0 MB
Max GPU memory usage: 0.0 MB


In [30]:
len(train_path), len(train_label)

(31803, 31803)

In [31]:
train_dataset = CT_Dataset(img_path=train_path, img_labels=np.array(train_label))
val_dataset = CT_Dataset(img_path=valid_path, img_labels=np.array(valid_label))
test_dataset = CT_Dataset(img_path=test_path, img_labels=np.array(test_label))

### Training

In [32]:
batch_size = 32
epoch = 50


# Train the ResNet101 model
model_kernel = ResNet50(num_classes=7)
# model_kernel = ResNet101(num_classes=7)
# model_kernel = EfficientNetB(num_classes=7)
# model_kernel = EfficientNetB4(num_classes=7)
# model_kernel = EfficientNetB7(num_classes=7)
# model_kernel = VGG16(num_classes=7)




path_save_model = f'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/model_/{model_kernel.get_name()}_{epoch}'
hist_kernel, model_kernel = train_model(model_kernel, train_dataset, val_dataset, test_dataset, device, path_save_model, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)

c:\Users\Lucas\anaconda3\envs\torch000\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lucas\anaconda3\envs\torch000\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training Start:
Epoch:1 / 50, lr: 0.00020 train loss:1.92998, train acc: 0.15565, valid loss:1.94730, valid acc:0.09900
Epoch:2 / 50, lr: 0.00020 train loss:1.94592, train acc: 0.10532, valid loss:1.94027, valid acc:0.37500
Epoch:3 / 50, lr: 0.00020 train loss:1.94574, train acc: 0.37726, valid loss:1.93772, valid acc:0.37500
Epoch:4 / 50, lr: 0.00020 train loss:1.94568, train acc: 0.37726, valid loss:1.93658, valid acc:0.37500
Epoch:5 / 50, lr: 0.00010 train loss:1.94564, train acc: 0.37726, valid loss:1.93501, valid acc:0.37500
Epoch:6 / 50, lr: 0.00010 train loss:1.94563, train acc: 0.37726, valid loss:1.93505, valid acc:0.37500
Epoch:7 / 50, lr: 0.00010 train loss:1.94568, train acc: 0.35620, valid loss:1.93510, valid acc:0.37500
Epoch:8 / 50, lr: 0.00010 train loss:1.94563, train acc: 0.37726, valid loss:1.93489, valid acc:0.37500
Epoch:9 / 50, lr: 0.00010 train loss:1.94566, train acc: 0.37726, valid loss:1.93490, valid acc:0.37500
Epoch:10 / 50, lr: 0.00005 train loss:1.94560, t

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [ ]:
56165dvsdvds

In [94]:
pred = torch.tensor([5, 5, 3, 5, 5, 5, 1, 2, 5, 5, 1, 2, 1, 5, 5, 2, 5, 2, 5, 2, 5, 5, 5, 5,
        5, 5, 5, 5, 2, 2, 5, 5], device='cuda:0')
labels = torch.tensor([1., 0., 0., 1., 2., 5., 1., 0., 1., 0., 2., 6., 4., 1., 4., 1., 0., 1.,
        1., 4., 0., 0., 3., 1., 1., 0., 3., 0., 4., 0., 3., 1.],
       device='cuda:0')
outputs = torch.tensor([[ 0.1039,  0.0196,  0.1340, -0.0606, -0.0624, -0.1492,  0.1489],
        [ 0.1638,  0.2615,  0.1425, -0.0664, -0.0220, -0.1176, -0.0086],
        [ 0.0499,  0.1659,  0.0624, -0.1114, -0.0556, -0.0509,  0.0630],
        [ 0.2102,  0.1398,  0.1844, -0.1581, -0.1947, -0.1324, -0.0041],
        [ 0.1990,  0.1115,  0.0485, -0.0643, -0.0910, -0.1942,  0.0245],
        [ 0.1179, -0.0302,  0.0422, -0.0449, -0.1007, -0.0709, -0.0055],
        [ 0.1428,  0.0111,  0.1842, -0.0272, -0.0348, -0.0723,  0.0479],
        [ 0.3330,  0.0644, -0.0239, -0.0471, -0.0416, -0.0922, -0.0870],
        [ 0.1566,  0.0252,  0.0238,  0.0467, -0.1089, -0.1529,  0.1591],
        [ 0.2148,  0.2172,  0.1216, -0.0358, -0.1598, -0.0973, -0.0901],
        [ 0.1118,  0.1927,  0.0710, -0.0861, -0.0437, -0.1435,  0.1106],
        [ 0.2535,  0.1214,  0.1725, -0.0893, -0.0666, -0.0366, -0.0411],
        [ 0.0763,  0.1014,  0.1511, -0.0185, -0.1423, -0.0700, -0.0263],
        [ 0.0939,  0.0642,  0.1573, -0.1136, -0.0953, -0.1442, -0.0850],
        [ 0.1151,  0.1056,  0.0030, -0.0148, -0.0330, -0.1542, -0.0428],
        [ 0.1255,  0.0819,  0.2538, -0.1424,  0.0214, -0.2399, -0.0481],
        [ 0.1598,  0.1540,  0.1070, -0.0325, -0.0415, -0.0631,  0.0029],
        [ 0.1825,  0.0751, -0.0437, -0.0999, -0.1244, -0.1818,  0.1489],
        [ 0.0868,  0.2163,  0.0354, -0.1322, -0.1326, -0.2135,  0.1156],
        [ 0.1741,  0.1908,  0.0927, -0.0541,  0.0426, -0.1062, -0.0114],
        [ 0.1124,  0.1096,  0.0362, -0.0808, -0.1658, -0.0545, -0.0318],
        [ 0.0577,  0.3386,  0.1067, -0.1172,  0.0819, -0.1923, -0.0056],
        [ 0.1731,  0.0573,  0.1518, -0.1144, -0.1122, -0.2052,  0.0473],
        [ 0.2782,  0.1307, -0.0476, -0.0025, -0.1064, -0.1131,  0.0086],
        [ 0.0075,  0.1485, -0.0037, -0.0975, -0.1250, -0.1801,  0.0208],
        [ 0.1043,  0.0389,  0.0586, -0.0291, -0.0548, -0.0643,  0.0337],
        [ 0.2485,  0.1398, -0.0677,  0.0053, -0.0031, -0.0763,  0.0114],
        [ 0.1488,  0.1722,  0.1780, -0.0195,  0.0364, -0.0952, -0.0126],
        [ 0.1048,  0.2050,  0.1818, -0.1468, -0.1223, -0.1630,  0.0551],
        [ 0.2036,  0.1393,  0.0089, -0.0018, -0.0506, -0.1196, -0.0014],
        [ 0.1451,  0.1543,  0.1173, -0.0693, -0.0077, -0.1841,  0.1808],
        [ 0.0967,  0.1962,  0.0779, -0.0615, -0.0854, -0.1139, -0.0776]],
       device='cuda:0')

In [95]:
predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
# predictions

In [96]:
y_pred = np.array(pred.cpu().numpy())
# y_pred

In [97]:
y_true = np.array(labels.cpu().numpy())
y_pred = np.array(pred.cpu().numpy())

In [98]:
y_true.shape, y_pred.shape

((32,), (32,))

In [99]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
# y_true = np.array([[1, 0, 1],
#                    [0, 1, 0]])
# y_pred = np.array([[1, 0, 0],
#                    [0, 1, 1]])
multilabel_confusion_matrix(y_true, y_pred)

array([[[22,  0],
        [10,  0]],

       [[19,  2],
        [10,  1]],

       [[23,  7],
        [ 2,  0]],

       [[28,  1],
        [ 3,  0]],

       [[28,  0],
        [ 4,  0]],

       [[11, 20],
        [ 0,  1]],

       [[31,  0],
        [ 1,  0]]], dtype=int64)

In [100]:
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

c:\Users\Lucas\anaconda3\envs\torch000\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# hist_kernel = hist_ResNet50
# model_kernel = model_ResNet50
label_model = model_kernel._get_name()

In [ ]:
torch.save(model_kernel.state_dict(), f'{path_save_model}.pth')

## Metricis

In [ ]:
def plot_metrics(hist, label_model) :
    # plot training curves
    epochs = range(1, len(hist['train_loss']) + 1)

    train_loss = [t.cpu().detach().numpy() for t in hist['train_loss']]
    val_loss = [t.cpu().detach().numpy() for t in hist['val_loss']]


    fig, ax = plt.subplots(1,2, figsize=(20,6))
    ax[0].plot(epochs, train_loss, 'r-', label='Train')
    ax[0].plot(epochs, val_loss, 'b-', label='Evaluation')
    ax[0].set_title('Loss')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    plt.savefig(f'{path_save_model}.png')  # This saves the plot as a PNG image

    ax[1].plot(epochs, hist['train_acc'], 'r-', label='Train')
    ax[1].plot(epochs, hist['val_acc'], 'b-', label='Evaluation')
    ax[1].set_title('Accuracy')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Acc')
    ax[1].legend()
    plt.savefig(f'{path_save_model}.png')  # This saves the plot as a PNG image
    

    plt.show()
    
    
plot_metrics(hist_kernel, label_model)

In [ ]:
def calculate_metrics(model, test_dataset, device, plot_images=False):
    # Initialize variables to store predictions and ground truth
    y_true = []
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    cont = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            cont+=1
            if cont == 5:
                break

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)


    return y_true, y_pred



y_true, y_pred = calculate_metrics(model_kernel, test_dataset, device, plot_images=True)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, precision_score, accuracy_score, recall_score, f1_score


In [ ]:
y_true, y_pred

In [ ]:
cm_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
# Calculate evaluation metrics
cm_matrix = multilabel_confusion_matrix(y_true, y_pred).ravel()
precision = precision_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
specificity = tn / (tn + fp)
sensitivity = tp / (fn + tp)

metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'accuracy': accuracy,
    'specificity': specificity,
    'sensitivity': sensitivity,
    'false negativo': fn
}


In [ ]:
def calculate_metrics(model, test_dataset, device, plot_images=False):
    # Initialize variables to store predictions and ground truth
    y_true = []
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    cont = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            cont+=1
            if cont == 5:
                break

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate evaluation metrics
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = precision_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    specificity = tn / (tn + fp)
    sensitivity = tp / (fn + tp)

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'specificity': specificity,
        'sensitivity': sensitivity,
        'false negativo': fn
    }

    return metrics



model_path = f'{path_save_model}.pth'

if label_model[:7] == "ResNet1":
    model_kernel = ResNet101(num_classes=7)

elif label_model[:7] == "ResNet5":
    model_kernel = ResNet50(num_classes=7)

elif label_model[:14] == "EfficientNetB0":
    model_kernel = EfficientNetB0(num_classes=7)
    
elif label_model[:14] == "EfficientNetB4":
    model_kernel = EfficientNetB4(num_classes=7)

elif label_model[:14] == "EfficientNetB7":
    model_kernel = EfficientNetB7(num_classes=7)

elif label_model[:5] == "VGG16":
    model_kernel = VGG16(num_classes=7)


model_kernel.load_state_dict(torch.load(model_path))
model_kernel.to(device)  # Move the model to the specified device
model_kernel.eval()  # Set the model to evaluation mode

# Call the function with plot_images=True to plot images
metrics = calculate_metrics(model_kernel, test_dataset, device, plot_images=True)
metrics

In [ ]:
y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
multilabel_confusion_matrix(y_true, y_pred)



In [ ]:
pd.DataFrame(metrics.items(), columns=['Metric', 'Value']).to_csv(f'{path_save_model}.csv')

In [ ]:
def process_images(model, test_dataset, device, plot_images=False, num_images_to_plot=10):
    # Initialize variables to store predictions and ground truth
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    images_plotted = 0  # Counter for the number of images plotted

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_pred.extend(predictions)

            # Plot images with true and predicted labels
            if plot_images and images_plotted < num_images_to_plot:
                plot_image(images, labels, predictions)
                images_plotted += 1

            # Break the loop if the desired number of images is plotted
            if images_plotted >= num_images_to_plot:
                break


def plot_image(images, true_label, predicted_label):
    if len(images.shape) == 4 and images.shape[1] == 3:
        # Convert CUDA tensor to numpy array and rearrange dimensions for RGB image
        images = images.cpu().detach().numpy().squeeze().transpose((1, 2, 0))
    else:
        # Convert CUDA tensor to numpy array and squeeze if necessary
        images = images.cpu().detach().numpy().squeeze()

    if len(images.shape) == 2:
        plt.imshow(images, cmap='gray')
    else:
        plt.imshow(images)
    
    # Format the labels
    true_label = true_label.item() if isinstance(true_label, torch.Tensor) else true_label
    predicted_label = predicted_label.item() if isinstance(predicted_label, torch.Tensor) else predicted_label
    
    plt.title(f'True label: {true_label}; Predicted: {predicted_label}')
    plt.show()
    
    
process_images(model_kernel, test_dataset, device, plot_images=True)

**Sugestão de Próximos Passos**

1. Colocar para salvar o modelo com a menor validation loss.
2. Colocar código para carregar o modelo salvo e fazer predições em imagens do dataset.
3. Criar métricas de acurácia (Accuracy, Precision, Recall, F-score, etc.) para avaliar o dataset de teste com o modelo salvo.

Quando tudo estiver pronto:
4. Criar novos modelos CNN (começar pela ResNet101).
5. Implementar outros modelos.

Quando estiver pronto:
6. Implementar no seu dataset.


Para binário tem que mudar a loss (CrossEntropy), a sigmoid da layer final para softmax

# Materiais e Métodos

## 1. Datasets

Explicação sobre os conjuntos de dados utilizados.

## 2. Modelos

Explicação sobre como funciona um modelo CNN (Convolutional Neural Network) e os parâmetros utilizados.

### 2.1. EfficientNet

### 2.2. ResNet101

...

## 3. Métricas
